In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

In [2]:
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.constant_initializer(1.0))
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    # print(type(conv))
    return tf.nn.relu(conv + biases)

In [3]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 1, 32], [32])
        
    # just a test
#     with tf.variable_scope("conv1",reuse=True):
#         # reuse is allowed, so load relu1_test from relu1
#         relu1_test = conv_relu(input_images, [5, 5, 1, 32], [32])
#         assert relu1_test == relu1
        
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        relu2 = conv_relu(relu1, [5, 5, 32, 64], [64])
        return relu2

In [4]:
def main(_):
    # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    with tf.variable_scope("image_filters") as scope:
        result1 = my_image_filter(x_image[0:1])
        scope.reuse_variables()
        result2 = my_image_filter(x_image[1:2])
    #data = x_image
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        batch_x,batch_y = mnist.train.next_batch(50)
        #print result1.eval(feed_dict={x_image:batch_x})
        #result = data[0:1].eval(feed_dict={x:batch_x})
        r1 = result1.eval(feed_dict={x:batch_x})
        r2 = result2.eval(feed_dict={x:batch_x})
        print(r1.shape)
        print(type(result1))
        print(type(r1))
        print(r2.shape)
    


In [5]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_dir', type=str,
                        default='/home/hadoop/workspace/tf_zero2one/data/mnist',
                        help='Directory for storing input data')
    FLAGS, unparsed = parser.parse_known_args()
    print(FLAGS,unparsed)
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Namespace(data_dir='/home/hadoop/workspace/tf_zero2one/data/mnist') ['-f', '/run/user/1001/jupyter/kernel-29158a07-5b51-450d-b219-759a6f4e05cd.json']
Extracting /home/hadoop/workspace/tf_zero2one/data/mnist/train-images-idx3-ubyte.gz
Extracting /home/hadoop/workspace/tf_zero2one/data/mnist/train-labels-idx1-ubyte.gz
Extracting /home/hadoop/workspace/tf_zero2one/data/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/hadoop/workspace/tf_zero2one/data/mnist/t10k-labels-idx1-ubyte.gz
(1, 28, 28, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
<type 'numpy.ndarray'>
(1, 28, 28, 64)


SystemExit: 

/home/hadoop/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
print(__doc__)

Automatically created module for IPython interactive environment


In [7]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
assert v1 is v,'not the same shared variable'

AssertionError: same shared variable

In [9]:
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        print(other_scope.name)
        # here we pass the foo_scope object rather than the name "foo"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"  # Not changed.

bar/baz
